# Language Processing with spaCy

## Installation and Basic Use

## Contents

1. Installation
2. Basic uses
3. Project: Creating a vocabulary list from words

## Installation

1. Installing spaCy
2. Installing models
3. Loading spaCy

#### spaCy

You should have Python 3 installed. Dependencies for spaCy may take a while to install.

pip install -U spacy

#### Models

SpaCy relies on installed language models. Before we do anything, we have to install at least one language model. We will install the medium English (91MB) and small German models (10MB).

python -m spacy download en_core_web_md

python -m spacy download de_core_news_sm

#### Loading spaCy

spaCy is now ready to load in Python. Let's load the English model.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_md')

## Basic Uses

1. Loading a text
2. Lemma, parts of speech and stopwords
3. Assessing similarity
4. Working with entities

#### Loading a text

Feeling festive, I chose two short sections of *A Christmas Carol* by Charles Dickens to analyze with spaCy. We will open them and process them with spaCy.

In [ ]:
bookLoc1 = 'xmascarol1.txt'
bookLoc2 = 'xmascarol2.txt'

with open(bookLoc1, 'r') as f:
    text1 = f.read()
    
with open(bookLoc2, 'r') as f:
    text2 = f.read()
    
doc1 = nlp(text1)
doc2 = nlp(text2)

#### Identifying Lemma, parts of speech and stopwords

You can iterate through a spaCy document object for its tokens. The token objects allow identification of things like the word lemma, its part of speech, its shape and whether it is a common word (a stopword). Let's take the first part of the text and interate through the tokens for:

1. lemma (stemmed word)
2. part of speech (e.g., noun)
3. syntactic dependecy (what it relates to in the sentence)
4. shape (capitalization)
5. stopword (whether it is a common word)

In [ ]:
for token in doc1:
    print(token.text,
         token.lemma_,
         token.pos_,
         token.dep_,
         token.shape_,
         token.is_stop)

#### Linguistic Similarity

What if we want to see if texts are similar to one another? Out of the box, spaCy can compare texts based on linguistic features. Let's take the two parts of *A Christmas Carol*.

In [ ]:
doc1.similarity(doc2)

Not surprisingly, these two short texts are pretty similar. Let's load part of a series of stories that Dickens published anonymously as Boz and is compiled as *Sketches by Boz, Illustrative of Every-Day Life and Every-Day People* and test for similarity.

In [ ]:
bozLoc = 'sketchesboz.txt'
    
with open(bozLoc,'r',encoding='utf8') as f:
    bozText = f.read()

bozDoc = nlp(bozText)

doc1.similarity(bozDoc)

Only slightly less similar than the two parts of *A Christmas Carol*. Let's try Dickens against parts of Joyce's *Ulysses*.

In [ ]:
ulyssesLoc = 'ulysses.txt'

with open(ulyssesLoc, 'r',encoding='utf8') as f:
    ulyssesText = f.read()

ulyssesDoc = nlp(ulyssesText)

bozDoc.similarity(ulyssesDoc)

Less similar although also very similar. What if we try against something very different, [an article about the NBA finals from fivethirtyeight.com](https://fivethirtyeight.com/features/warriors-raptors-game-2-nba-finals/):

In [ ]:
basketballLoc = 'basketball.txt'

with open(basketballLoc, 'r',encoding='utf8') as f:
    basketballText = f.read()

basketballDoc = nlp(basketballText)

bozDoc.similarity(basketballDoc)

Less similar still, although still pretty similar. A different model or a trained model might give more differentiating results.

#### Identifying Entities

A useful feature in spaCy is its ability to distinguish entities. Entities are properly named people or places. Out of the box, spaCy does a pretty good job of finding names, institutions and locations. We can iterate through the entities in the text just like tokens:

In [ ]:
for ent in basketballDoc.ents:
    print(ent.text,ent.label_)

There are some problems with this index, of course. "Six dimes" is slang for six assists, for example. If you were interested in making a gazeteer (an place index), it would be easy to do by only printing those entities that have geographical qualities. These are "geographical or political entities" (GPE) or "locations" (LOC)

In [ ]:
for ent in basketballDoc.ents:
    if ent.label_ in ['GPE','LOC']:
        print(ent.text,ent.label_)

### Project: Creating a List of Words for Language Study

1. Eliminating unwanted tokens
2. Creating an updating set

Let's say you are teaching an ESL class for beginners. There are many texts you could assign, but how can you make a vocabulary list? And once you have a list for one text, how can you make sure the words do not repeat?

Let's make a set of the words we need to learn and that we have already learned:

In [ ]:
words2learn = set()
learnedWords = set()

#### Eliminating unwanted tokens

We need to iterate through the list of tokens from our document and make sure that easy words or non-words do not end up in the list. The first thing to do is to eliminate tokens that should not be in the list. These include punctuation (PUNCT), proper nouns (PROPN), spaces (SPACE), numbers (NUM) and symbols (SYM). Simply test to see if the tokens part of speech is one of these parts. We also should test to see if the token is a stopword so that overly simple words do not end up on the list.

Here we iterate through the first part of *A Christmas Carol* and add the lemma and its part of speech to our learning list. Doing so will mean that strings that could represent different parts of speech (e.g., "run") will both end up in the list.

In [ ]:
for token in doc1:
    if token.pos_ not in ['PUNCT','PROPN','SPACE','NUM','SYM'] and not token.is_stop:
        words2learn.add((token.lemma_,token.pos_))

for word in words2learn:
    print(word[0],word[1])

Then it might make sense to write up this part of the code as a function so we can use it again. 

In [ ]:
def usableWords(doc):
    w2l = set()
    for token in doc:
        if token.pos_ not in ['PUNCT','PROPN','SPACE','NUM','SYM'] and not token.is_stop:
            w2l.add((token.lemma_,token.pos_))
    return w2l

Let's assume the student learned the words from the first part of *A Christmas Carol* and wants to learn new words from the second part. We can put the learned words in that set and figure out what words need to be learned.

In [ ]:
learnedWords = learnedWords|words2learn

newWords = usableWords(doc2)

words2learn.clear()

print(newWords-learnedWords)


#### Creating a set that updates

It might be useful to take a corpus of texts. There is a site called ESL Fast that has short, easy texts. Using the  urllib module and the HTML parsing module BeautifulSoup, we can download these texts and make lists from them. If you need to download BeautifulSoup, do so in your terminal:

pip install beautifulsoup4

Let's import these modules and download the first fourteen "supereasy" texts on the site. If you don't know how to use BeautifulSoup, you might look at [The Programming Historian's lesson](https://programminghistorian.org/en/lessons/intro-to-beautiful-soup).

In [ ]:
import bs4




In [ ]:
import urllib
from bs4 import BeautifulSoup

texts = []

for story in range(1,15):
    url = 'https://www.eslfast.com/supereasy/se/supereasy' + ('00'+str(story))[-3:] + '.htm'
    page = urllib.request.urlopen(url).read()
    text = BeautifulSoup(page, "html.parser").find(class_='MsoNormal').text
    texts.append(text)
    
print(texts[0])

Let's make a function that takes our learned words plus our new text and generates a list of words we need to learn and prints the words to learn, and one that adds the learned words:

In [ ]:
def tolearn(oldWords, newDoc):
    words = usableWords(newDoc)
    newWords = words-oldWords
    for word in newWords:
        print(word[0],'('+word[1]+')')
    return newWords

def learned(oldWords, newWords):
    return oldWords|newWords

Then we can iterate through our texts, get the words to learn for each lesson:

In [ ]:
learnedWords = set()

for text in texts:
    print('\n***LESSON '+str(texts.index(text)+1)+'***')
    textDoc = nlp(text)
    words2learn = tolearn(learnedWords,textDoc)
    learnedWords = learned(learnedWords,words2learn)